# 딥러닝 실습 환경 소개

In [1]:
import tensorflow as tf

## 하이퍼파라미터 설정

In [16]:
EPOCHS = 5
BATCH_SIZE = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (28, 28, 1)
num_classes = 10

## 데이터셋 가져오고, 정리하기
- 데이터 정규화를 위해 255.0을 나눈다(Rescale).
- mnist 데이터의 shape은 (file_size, width, height)


#### Rescale시 주의사항
- traing에서만 변환만 하고 test는 하지 않해도 된다.
- 하지만 rescale은 train와 test 모두 적용해줘야한다.

In [15]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

## 데이터셋 Dimension 변경
- CNN Model의 input data의 dim은 batch, width, height, channel 이다.
- Dataset의 마지막 차원에 channel을 추가한다.

In [ ]:
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

## Dataset shuffle
- Data shuffle을 수행하면 데이터를 무작위로 섞어 
- 분산의 크기를 작게하고, 데이터를 좀 더 일반적이고 overfitting을 줄여줍니다.
- shuffle은 다소 시간이 오래걸린다.

reference: https://datascience.stackexchange.com/questions/24511/why-should-the-data-be-shuffled-for-machine-learning-tasks

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

## 네트워크 구조 정의
- keras를 이용해서 network를 정의합니다.
- mnist의 데이터의 shape이 28 * 28 이므로 input_shape을 28 * 28로 설정합니다.
- Layer의 activation function은 sigmoid를 사용합니다.
- 다분류를 위해 softmax를 사용합니다.

In [3]:
# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 루프 정의

In [5]:
# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 네트워크 생성

In [13]:
model = MyModel()

## 손실함수, 최적화 알고리즘 정의

In [8]:
# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 알고리즘 평가지표 설정 - Define performance metrics

In [14]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 알고리즘 학습, 테스트 루프 - Do training loop and test

In [10]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.3964349031448364, Accuracy: 89.60333251953125, Test Loss: 0.2231641560792923, Test Accuracy: 93.51000213623047
Epoch 2, Loss: 0.19603028893470764, Accuracy: 94.41666412353516, Test Loss: 0.16395817697048187, Test Accuracy: 95.27000427246094
Epoch 3, Loss: 0.14371120929718018, Accuracy: 95.83499908447266, Test Loss: 0.13173457980155945, Test Accuracy: 96.13999938964844
Epoch 4, Loss: 0.11228719353675842, Accuracy: 96.79833221435547, Test Loss: 0.11388211697340012, Test Accuracy: 96.66000